# 第12章　Pandas：経済データ分析II

## はじめに

In [ ]:
%pip install -q japanize-matplotlib py4macro
import japanize_matplotlib
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import py4macro

## バブル景気前後の成長率の比較

コード12.2.1

In [ ]:
jp = py4macro.data("jpn-yr")
jp.columns

コード12.2.2

In [ ]:
jp.iloc[:3, [0]]

コード12.2.3

In [ ]:
jp.loc[:"1982", ["gdp"]]

コード12.2.4

In [ ]:
jp["gdp_log"] = np.log( jp["gdp"] )
jp["gdp_growth"] = jp["gdp_log"].diff()

g_before = jp.loc[:"1992", "gdp_growth"].mean()
g_after = jp.loc["1993":, "gdp_growth"].mean()
g_diff = g_before - g_after

print(f"バブル崩壊前：{g_before:.4f}")
print(f"バブル崩壊後：{g_after:.4f}")
print(f"バブル崩壊前後での差：{g_diff:.4f}")

## 消費 vs 投資

コード12.3.1

In [ ]:
cols = ["gdp", "consumption", "investment"]

for col in cols[1:]:
    jp[col+"_growth"] = np.log( jp[col] ).diff()

コード12.3.2

In [ ]:
lstyles = ["-", ":", "--"]
labels = ["GDP", "消費", "投資"]

for col, lstyle, label in zip( cols, lstyles, labels):
    plt.plot(col+"_growth", data=jp, color="black",
             linewidth=1, linestyle=lstyle, label=label)

plt.legend()
plt.show()

コード12.3.3

In [ ]:
print("成長率の標準偏差（GDP=1）")
print("-"*24)

for col, label in zip(cols[1:], labels[1:]):
    gdp_std = jp["gdp_growth"].std()
    std = jp[col+"_growth"].std() / gdp_std
    print(f"{label}: {std:.4f}")

コード12.3.4

In [ ]:
plt.scatter("gdp_growth", "consumption_growth", data=jp,
            alpha=0.7, color="black", label="消費",
            marker="x")
plt.scatter("gdp_growth", "investment_growth", data=jp,
            alpha=0.3, color="black", label="投資")
plt.xlabel("GDPの成長率")
plt.ylabel("消費，投資の成長率")
plt.legend()
plt.show()

コード12.3.5

In [ ]:
vcov = jp[["gdp_growth", "consumption_growth"]].cov()
vcov

コード12.3.6

In [ ]:
print(vcov.iloc[0, 1])

コード12.3.7

In [ ]:
print("成長率：GDPとの共分散")
print("-"*21)

for col, label in zip( cols[1:], labels[1:]):
    cvar = jp[[col+"_growth", "gdp_growth"]].cov().iloc[0,1]
    print(f"{label}: {cvar:.6f}")

コード12.3.8

In [ ]:
ccorr = jp[["gdp_growth", "consumption_growth"]].corr()
ccorr

コード12.3.9

In [ ]:
print("成長率：GDPとの相関係数")
print("-"*20)

for col, label in zip( cols[1:], labels[1:]):
    cc = jp[[col+"_growth", "gdp_growth"]].corr().iloc[0,1]
    print(f"{label}: {cc:.6f}")

## 1850年以降の経済成長率

コード12.4.1

In [ ]:
mad = py4macro.data("mad")
mad.columns

コード12.4.2

In [ ]:
countries = ["Japan", "Germany", "United States", "China"]
kuni4 = {}

for country in countries:
    cond1 = ( mad["country"]==country )
    cond2 = ( mad["year"] >= 1850 )
    cond = cond1 & cond2
    tmp = mad.loc[cond, ["year", "gdppc"]]
    tmp["gdppc_log"] = np.log( tmp["gdppc"] )
    kuni4[country] = tmp

`tmp`を使わずに，次のコードでも同じ結果となります。

In [ ]:
countries = ["Japan", "Germany", "United States", "China"]
kuni4 = {}

for country in countries:
    cond1 = ( mad["country"]==country )
    cond2 = ( mad["year"] >= 1850 )
    cond = cond1 & cond2
    kuni4[country] = mad.loc[cond, ["year", "gdppc"]]
    kuni4[country]["gdppc_log"] = np.log( kuni4[country]["gdppc"] )

コード12.4.3

In [ ]:
kuni4["Japan"].head(1)

コード12.4.4

In [ ]:
labels = ["日本", "ドイツ", "米国", "中国"]
lstyles = ["-", "--", "-.", ":"]

for country, label, lstyle in zip( countries, labels, lstyles ):
    plt.plot("year", "gdppc_log", data=kuni4[country],
             color="black", label=label, linestyle=lstyle)

plt.legend()
plt.title("一人当たりGDP（対数）", size=13)
plt.show()

コード12.4.5

In [ ]:
dfjp = kuni4["Japan"]
dfjp["growth"] = dfjp["gdppc"].pct_change()

cond45 = ( dfjp["year"]==1945 )
cond46 = ( dfjp["year"]==1946 )

growth45 = dfjp.loc[cond45,"growth"].iloc[0]
growth46 = dfjp.loc[cond46,"growth"].iloc[0]

print("日本：一人当たりGDP")
print("-"*30)
print(f"1944 → 1945年の変化率： {growth45:.2%}" )
print(f"1945 → 1946年の変化率： {growth46:.2%}" )

コード12.4.6

In [ ]:
print("日本の一人当たりGDP成長率")
print("-"*21)

for i in range(7):
    start = 1950 + 10 * i
    end = start + 10
    cond = ( ( start <= dfjp["year"] ) &
             ( dfjp["year"] <= end   ) )
    g = dfjp.loc[cond,"gdppc_log"].diff().mean()
    print(f"{start}年代：{g:.2%}")

## 超長期の経済成長：日本と英国

コード12.5.1

In [ ]:
countries = ["Japan", "United Kingdom"]
cols = ["year", "gdppc"]

kuni2 = {}

for country in countries:
    cond = ( mad["country"]==country )
    kuni2[country] = mad.loc[cond, cols]

kuni2["Japan"].head()

コード12.5.2

In [ ]:
labels = ["日本", "英国"]
lstyles = ["-", ":"]

for country, label, lstyle in zip(countries, labels, lstyles ):
    plt.plot("year", "gdppc", data=kuni2[country],
             color="black", label=label, linestyle=lstyle)
plt.title("一人当たりGDP", size=13)
plt.legend()
plt.show()

## 世界の所得格差

コード12.6.1

In [ ]:
pwt = py4macro.data("pwt")

コード12.6.2

In [ ]:
cols = ["country", "rgdpe", "pop"]
cond = ( pwt["year"]==2019 )
pwt = pwt.loc[cond, cols]

pwt["gdppc"] = pwt["rgdpe"] / pwt["pop"]
pwt.head()

コード12.6.3

In [ ]:
cond = ( pwt["country"]=="Japan" )
gdppc_jp = pwt.loc[cond, "gdppc"].iloc[0]
pwt["gdppc_jp"] = pwt["gdppc"] / gdppc_jp

num = pwt["gdppc_jp"].nunique()

plt.hist("gdppc_jp", data=pwt, color="black",
         bins=17, edgecolor="white")
plt.title(f"一人当たり実質GDPの分布（{num}ヶ国，2019年）",
          size=13)
plt.ylabel("国の数", size=11)
plt.show()

コード12.6.4

In [ ]:
n = ( pwt["gdppc_jp"] >= 1.0 ).sum()
print(f"日本は{n}番目です。")

コード12.6.5

In [ ]:
n = ( pwt["gdppc_jp"] <= 0.1 ).sum()
print(f"0.1以下：{n}ヶ国")

n = ( pwt["gdppc_jp"] <= 0.05 ).sum()
print(f"0.05以下：{n}ヶ国")